In [2]:
#43721
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
pd_xela_allfiles = pd.read_csv('data.csv', index_col=0)
pd_sliplabel_allfiles = pd.read_csv('labels.csv', index_col=0)

data = pd_xela_allfiles.to_numpy()
labels = pd_sliplabel_allfiles.to_numpy()

In [8]:
sc = MinMaxScaler()
sc.fit(data)
data = sc.transform(data)

229642

In [5]:
data_prime = []
for t in range(data.shape[0]-9):
    data_prime.append(data[t:t+10])
data_prime = np.array(data_prime)

In [5]:
labels = labels[9:]

In [6]:
data_prime.shape, labels.shape

((229642, 10, 48), (229642, 1))

In [7]:
n_v = 48
n_a = 48
tx = 10
reshaper = Reshape((1, 48))                  
LSTM_cell = LSTM(n_a, return_state = True)         
densor = Dense(1, activation='sigmoid')  

X = Input(shape=(tx, n_v)) 
a0 = Input(shape=(n_a,), name='a0')
c0 = Input(shape=(n_a,), name='c0')
a = a0
c = c0

outputs = []

for t in range(tx):
    x = X[:, t, :]
    x = reshaper(x)
    a, _, c = LSTM_cell(x,[a, c])
out = densor(a)

model = Model(inputs=[X, a0, c0], outputs=out)

In [8]:
opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
m = labels.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [10]:
history = model.fit([data_prime, a0, c0], labels, epochs=3, validation_split=0.1)

Epoch 1/3
6459/6459 [==============================] - 171s 21ms/step - loss: 0.0955 - accuracy: 0.9583 - val_loss: 0.5235 - val_accuracy: 0.9395
Epoch 2/3
6459/6459 [==============================] - 112s 17ms/step - loss: 0.0628 - accuracy: 0.9726 - val_loss: 0.5239 - val_accuracy: 0.9444
Epoch 3/3
6459/6459 [==============================] - 110s 17ms/step - loss: 0.0566 - accuracy: 0.9757 - val_loss: 0.5268 - val_accuracy: 0.9459


In [11]:
# model.save('nelson_model.h5')